#                        Projet-4 (Analysez les ventes de votre entreprise)

# Sommaire

# Mission 1: Nettoyage des données
1. Customer
2. Products
3. Transactions

# Mission 2 : Analyse des données
1. des indicateurs de tendance centrale et de dispersion ;

2. une analyse de concentration, via une courbe de Lorenz et un indice de Gini ;

3. des représentations graphiques, dont au moins un histogramme, une représentation avec des "boîtes à moustaches" ;

4.  une représentation de série temporelle (c’est-à-dire un graphique dont l’axe des abscisses représente des dates) ;

4. des analyses bivariées.

# Mission 3: Étude des Corrélations
1. corrélation entre le sexe des clients et les catégories de produits achetés 

2. une corrélation entre l'âge des clients et le montant total des achats ;

3. une corrélation entre l'âge des clients et la fréquence d’achat (ie. nombre d'achats par mois par exemple) ;

4. une corrélation entre l'âge des clients et la taille du panier moyen (en nombre d’articles) ;

5. une corrélation entre l'âge des clients et les catégories de produits achetés.

In [1]:
import pandas as pd ; 
import numpy as np ; 
import seaborn as sns ; 
import datetime as dt ; 
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import arange, plot 
from matplotlib.pyplot import figure
import scipy.stats as st 
import statsmodels.api as sm 
import statsmodels.formula.api as smf
import warnings ; warnings.filterwarnings('ignore')

# Importer les fichiers de données.csv

In [2]:
df_transactions=pd.read_csv("transactions.csv",parse_dates=[1])
df_customers=pd.read_csv("customers.csv", dtype={'sex': 'category'})
df_produits=pd.read_csv("products.csv", dtype={'categ': 'category'})

# Mission 1: (Nettoyage des données)

# a. Description des données

In [3]:
df_transactions

,id_prod,date,session_id,client_id
0,0_1483,4/10/2021,s_18746,c_4450
1,2_226,2/3/2022,s_159142,c_277
2,1_374,9/23/2021,s_94290,c_4270
3,0_2186,10/17/2021,s_105936,c_4597
4,0_1351,7/17/2021,s_63642,c_1242
...,...,...,...,...
337011,1_671,5/28/2021,s_40720,c_3454
337012,0_759,6/19/2021,s_50568,c_6268
337013,0_1256,3/16/2021,s_7219,c_4137
337014,2_227,10/30/2021,s_112349,c_5


In [4]:
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337016 entries, 0 to 337015
Data columns (total 4 columns):
id_prod       337016 non-null object
date          337016 non-null object
session_id    337016 non-null object
client_id     337016 non-null object
dtypes: object(4)
memory usage: 10.3+ MB


In [5]:
df_customers

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943
...,...,...,...
8618,c_7920,m,1956
8619,c_7403,f,1970
8620,c_5119,m,1974
8621,c_5643,f,1968


In [6]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8623 entries, 0 to 8622
Data columns (total 3 columns):
client_id    8623 non-null object
sex          8623 non-null category
birth        8623 non-null int64
dtypes: category(1), int64(1), object(1)
memory usage: 143.3+ KB


In [7]:
df_produits

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0
3,1_587,4.99,1
4,0_1507,3.99,0
...,...,...,...
3282,2_23,115.99,2
3283,0_146,17.14,0
3284,0_802,11.22,0
3285,1_140,38.56,1


In [8]:
df_produits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287 entries, 0 to 3286
Data columns (total 3 columns):
id_prod    3287 non-null object
price      3287 non-null float64
categ      3287 non-null category
dtypes: category(1), float64(1), object(1)
memory usage: 54.7+ KB


In [9]:
df_customers.head()

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943


In [10]:
df_customers.birth.unique()

array([1967, 1975, 1984, 1962, 1943, 1993, 1978, 1971, 1982, 1945, 2003,
       1959, 1977, 1954, 1987, 2000, 1992, 1963, 1958, 1994, 1936, 1986,
       1942, 1970, 1957, 1968, 2002, 2004, 1979, 1974, 1964, 1951, 1937,
       1981, 1965, 1960, 1996, 1983, 1990, 1955, 1988, 1991, 1972, 1980,
       1989, 1976, 1985, 1953, 1998, 1956, 1948, 1973, 1969, 1938, 1995,
       1952, 1949, 1999, 2001, 1939, 1950, 1966, 1935, 1941, 1961, 1997,
       1944, 1929, 1947, 1946, 1932, 1931, 1933, 1930, 1940, 1934],
      dtype=int64)

In [11]:
df_customers.birth.value_counts

<bound method IndexOpsMixin.value_counts of 0       1967
1       1975
2       1984
3       1962
4       1943
        ... 
8618    1956
8619    1970
8620    1974
8621    1968
8622    1982
Name: birth, Length: 8623, dtype: int64>

# b.  Ajouter la colonne " Age " dans les données

In [12]:
date = dt.datetime.now()
df_customers['age']= date.year-df_customers.birth.astype('int')

In [13]:
df_customers.head()

,client_id,sex,birth,age
0,c_4410,f,1967,54
1,c_7839,f,1975,46
2,c_1699,f,1984,37
3,c_5961,f,1962,59
4,c_5320,m,1943,78


# c. Mise en forme de la date dans le cadre de données 

In [14]:
df_transactions=df_transactions.loc[(df_transactions.client_id!='ct_0')&(df_transactions.client_id!='ct_1')]

def date_claire(date):
    return(date.split(".")[0])
df_transactions["date"]=df_transactions["date"].apply(date_claire)

df_transactions.rename(columns={"date": "date_et_heure"}, inplace=True)
df_transactions["date_et_heure"]=pd.to_datetime(df_transactions["date_et_heure"])


# La conversion de date_et_heure en datetime nous permet d'appliquer les fonctions get_date() et get_hour() sur cette
# variable nous renvoient respectivement la date et l'heure:

def get_date(dh):
    return(dh.date());
df_transactions["date"]=df_transactions["date_et_heure"].apply(get_date)

def get_hour(dh):
    return(dh.time());
df_transactions["heure"]=df_transactions["date_et_heure"].apply(get_hour)


df_transactions=df_transactions.drop(columns="date_et_heure")  
df_transactions

,id_prod,session_id,client_id,date,heure
0,0_1483,s_18746,c_4450,2021-04-10,00:00:00
1,2_226,s_159142,c_277,2022-02-03,00:00:00
2,1_374,s_94290,c_4270,2021-09-23,00:00:00
3,0_2186,s_105936,c_4597,2021-10-17,00:00:00
4,0_1351,s_63642,c_1242,2021-07-17,00:00:00
...,...,...,...,...,...
337011,1_671,s_40720,c_3454,2021-05-28,00:00:00
337012,0_759,s_50568,c_6268,2021-06-19,00:00:00
337013,0_1256,s_7219,c_4137,2021-03-16,00:00:00
337014,2_227,s_112349,c_5,2021-10-30,00:00:00


# d. Joindre les 2 tables (clients, produits) avec des transactions par merge

In [15]:
ventes=df_transactions.merge(df_produits, how='left',on='id_prod').merge(df_customers,how='left',on='client_id')
ventes.head()

,id_prod,session_id,client_id,date,heure,price,categ,sex,birth,age
0,0_1483,s_18746,c_4450,2021-04-10,00:00:00,4.99,0,f,1977,44
1,2_226,s_159142,c_277,2022-02-03,00:00:00,65.75,2,f,2000,21
2,1_374,s_94290,c_4270,2021-09-23,00:00:00,10.71,1,f,1979,42
3,0_2186,s_105936,c_4597,2021-10-17,00:00:00,4.20,0,m,1963,58
4,0_1351,s_63642,c_1242,2021-07-17,00:00:00,8.99,0,f,1980,41


In [16]:
ventes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336816 entries, 0 to 336815
Data columns (total 10 columns):
id_prod       336816 non-null object
session_id    336816 non-null object
client_id     336816 non-null object
date          336816 non-null object
heure         336816 non-null object
price         336713 non-null float64
categ         336713 non-null category
sex           336816 non-null category
birth         336816 non-null int64
age           336816 non-null int32
dtypes: category(2), float64(1), int32(1), int64(1), object(5)
memory usage: 22.5+ MB


# e. Maintenant, vérifie les données et je trouve que t_0=-1 dans id_produits n'a qu'une seule valeur négative, donc je laisse tomber 

In [17]:
ventes_1=ventes.drop(ventes[ventes.id_prod=='T_0'].index)

In [18]:
ventes_1.id_prod.count()

336816

# f. Remplir les valeurs NaN avec les valeurs médian de la catégorie 0

In [19]:
ventes_1.sort_values(by = 'price', ascending = False).tail()

,id_prod,session_id,client_id,date,heure,price,categ,sex,birth,age
322523,0_2245,s_16936,c_4167,2021-04-06,00:00:00,NaN,NaN,f,1979,42
329226,0_2245,s_13738,c_7790,2021-03-30,00:00:00,NaN,NaN,f,1983,38
330297,0_2245,s_128815,c_6189,2021-12-03,00:00:00,NaN,NaN,f,1984,37
335331,0_2245,s_26624,c_1595,2021-04-27,00:00:00,NaN,NaN,f,1973,48
336020,0_2245,s_28235,c_5714,2021-05-01,00:00:00,NaN,NaN,f,1972,49


In [20]:
ventes_1.isnull().sum(axis = 0)

id_prod         0
session_id      0
client_id       0
date            0
heure           0
price         103
categ         103
sex             0
birth           0
age             0
dtype: int64

In [21]:
ventes_1[ventes_1['id_prod']=='0_2245'].count()['id_prod']

103

# g. Le produit 0_2245 n'a pas de prix ni de catégorie associés.

Ce produit a été vendu 103 fois dans l'année.

Nous associons la catégorie 0 au produit et le prix médian des produits vendus entre 80 et 120 fois dans l'année

In [22]:
size=ventes_1.groupby('id_prod').size()
produit=size[(size>80)&(size<120)].index.values
prix=df_produits[df_produits.id_prod.isin(produit)]['price'].median()
ventes_1.loc[ventes_1.id_prod=='0_2245','categ']='0'
ventes_1.loc[ventes_1.id_prod=='0_2245','price']=prix

In [23]:
ventes_1.loc[ventes_1.id_prod=='0_2245'].head(5)

,id_prod,session_id,client_id,date,heure,price,categ,sex,birth,age
6231,0_2245,s_49705,c_1533,2021-06-17,00:00:00,12.99,0,m,1972,49
10797,0_2245,s_49323,c_7954,2021-06-16,00:00:00,12.99,0,m,1973,48
14045,0_2245,s_124474,c_5120,2021-11-24,00:00:00,12.99,0,f,1975,46
17480,0_2245,s_172304,c_4964,2022-02-28,00:00:00,12.99,0,f,1982,39
21071,0_2245,s_3,c_580,2021-03-01,00:00:00,12.99,0,m,1988,33


In [24]:
ventes_1.head()

,id_prod,session_id,client_id,date,heure,price,categ,sex,birth,age
0,0_1483,s_18746,c_4450,2021-04-10,00:00:00,4.99,0,f,1977,44
1,2_226,s_159142,c_277,2022-02-03,00:00:00,65.75,2,f,2000,21
2,1_374,s_94290,c_4270,2021-09-23,00:00:00,10.71,1,f,1979,42
3,0_2186,s_105936,c_4597,2021-10-17,00:00:00,4.20,0,m,1963,58
4,0_1351,s_63642,c_1242,2021-07-17,00:00:00,8.99,0,f,1980,41


# Maintenant, nous avons le tableau final avec des données nettoyées sans valeurs null/NaN.

In [26]:
ventes_1.to_csv("ventes_1.csv")